In [66]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
# @Time    : 2022/9/26 14:40
# @Author  : Wang Yujia
# @File    : data_extract_for_asc_symmetry.ipynb
# @Description : 为ascending-price auction从outcomes中提取features和data。ref：信息不对称的paper里symmetry的情况。

# 0. what for
1. 提取ascending-price auction（symmetry）的GT model需要的features, or settings
    - 并不是所有settings都会被considered into, 样本数小于`threshold`的settings不予考虑
2. 根据1中的settings，提取对应的data. 选择method-1计算`n`与`p`，作为target data使用
3. 代码参考自`../BasicInfo/calculate_n.np`，对于列名/参数名进行了优化和更改
4. Output：2个csv
    - `datawithnp_asc_symmetry.csv`：去除了重复行，但是未没经过threshold筛选的data
    - `datawithnp_asc_symmetry_selected.csv`去除了重复行，且经过threshold筛选，GTmodel需要的参数和target data都在这里了
5. **当GT模型计算需要更多的feature时**，更改`features_GT`就好
6. **检查逻辑的2个思路**：
    - 对于同一个unique setting, 是不是 sum(['cnt_n_1'])==['cnt_uniq'].
    - 对于同一个unique setting, 是不是['cnt_n_1']==2对应的['P']是['cnt_n_1']==1对应的['P']的 2倍
    - 逻辑检查 see 2.2.4
7. 刚意识到，'n'这个值其实是不会变的，变的是'p'，后者取决于我们把什么样的feature归为一类，而前者就是一场拍卖的轮数，是一个既已发生的客观值

# 1. Preparations
## 1.1 全局设置

In [67]:
# outcomes dataset
outcomes_orignal_path = "../data/outcomes.tsv"
# traces dataset
traces_original_path = "../data/traces.tsv"
# outcomes和traces共有的auction_id: `common_auction_id`
# Clearly这里选择method-1计算(n,p)
common_auction_id_path = "../data/common_auction_id.csv"
# 计算完n和p后，可以使用的dataset被output在
output_path = "../data/info_asymm/datawithnp_asc_symmetry.csv"
# 经过threshold后，得到的dataset
data_selected_path = "../data/info_asymm/datawithnp_asc_symmetry_selected.csv"

# 最终GT model需要的features（不包括np），其中`product_id` is not necessary
features_GT = ['auction_id','product_id','bidincrement','bidfee','retail']
# 衡量一场auction是否unique的标志
# 发现存在'product_id'相同的但'retail'不同的情况，也具有实际意义，设置为uniq setting，不过因为样本量少而清洗掉就是另外一回事了
unique_setting = ['product_id', 'bidincrement', 'bidfee','retail']
# 样本数小于`threshold`的settings不予考虑
threshold = 16

import numpy as np
import pandas as pd

## 1.2 读取data

In [68]:
outcomes = pd.read_csv(outcomes_orignal_path, sep='\t')
outcomes = outcomes[features_GT]
traces = pd.read_csv(traces_original_path, sep='\t')
common_auction_id = pd.read_csv(common_auction_id_path, encoding="utf-8")

# 2. 计算n和p
## 2.1 计算n
1. 这里计算n是“方法一”：just count according to `traces.tsv`

In [69]:
# 2.1.1 traces数据groupby，目的是为了数n，并且保留需要的列: trace_groupby_auctionid
trace_grouped = traces.groupby('auction_id')
trace_groupby_auctionid = trace_grouped.count()['bid_user'].to_frame()     # 这里count出来的'n_1'就是'duration'
trace_groupby_auctionid.reset_index(drop=False, inplace=True)              # 整理data，index是'auction_id'，把index变成一列
trace_groupby_auctionid.columns=['auction_id','n_1']

# 2.1.2 从outcomes中提取共有的auction_id对应的data: outcomes_incommon
# It turns out all 'auction_id' recorded in traces.tsv are also in outcomes.tsv
outcomes_incommon = outcomes[outcomes['auction_id'].isin(common_auction_id['auction_id'])]

# 2.1.3 通过左连接的方式，得到一个包括数据'n'的表：data_withn
data_withn = pd.merge(trace_groupby_auctionid, outcomes_incommon, on='auction_id', how="left")
data_withn.drop(['auction_id'],axis = 1,inplace=True)    #不需要这个信息
data_withn['product_id'] = data_withn['product_id'].astype(int)

In [70]:
# 2.1.4 New Added: drop data where ['flg_fixedprice'] == 1
# 按照方法一的话会发现，traces.tsv中没有任何['flg_fixedprice'] == 1的信息
# np.where(data_withn['flg_fixedprice'] == 1)

## 2.2 计算p
1. 实际上`unique setting=['product_id', 'bidincrement', 'bidfee','retail']`，unique setting一样的auction认为是同一个
2. 但是存在不同'product_id'具有同样的'retail'，这一点需要注意
3. 因此对于GT model而言，`unique setting for GT=['retail', 'bidincrement', 'bidfee']`
4. `P = cnt_n_1 / cnt_uniq`
5. cnt_uniq = ['n_1'].nunique() under the same 'unique setting'

In [71]:
# 2.2.1 each unique setting对应了'cnt_uniq'场auction: data_withn_cnt
# 注意'cnt_uniq'并不需要出现在最后的data中，但是后面和threshold会联动一下
data_grouped_tmp = data_withn.groupby(unique_setting,as_index=False)
tmp = pd.DataFrame(data_grouped_tmp.size())
tmp.reset_index(drop=False, inplace=True)    # 把index变成col

data_withn_cnt = pd.merge(data_withn, tmp, on=unique_setting, how="left")
data_withn_cnt.rename(columns={0: 'cnt_uniq'}, inplace=True)
data_withn_cnt.head()

,n_1,product_id,bidincrement,bidfee,retail,cnt_uniq
0,151,10012342,12,60,64.12,12
1,79,10012342,12,60,64.12,12
2,72,10012359,12,60,169.99,14
3,283,10012359,12,60,169.99,14
4,51,10011778,12,60,27.99,9


- P = cnt_n / cnt_uniq,上面已经算了' cnt_uniq'，下面需要算cnt_n

In [72]:
# 2.2.2 计算cnt_n_1，并添加到data_withn的一列: data_withn_cnt_n1
        # cnt_n_1表示某个setting下的n_1某数值出现了几次/ Example: cnt_n=2表示在某个setting下，有2场拍卖持续了n_1轮
features_cnt_n_1 = unique_setting+['n_1']
data_grouped_tmp = data_withn.groupby(features_cnt_n_1,as_index=False)
tmp = pd.DataFrame(data_grouped_tmp.size())
tmp.reset_index(drop=False, inplace=True)
data_withn_cnt_n1 = pd.merge(data_withn_cnt, tmp, on=features_cnt_n_1, how="left")
data_withn_cnt_n1.rename(columns={0: 'cnt_n_1'}, inplace=True)

- P = cnt_n / cnt_uniq
- 输出的结果在`data_withn_cnt_n12`中，其中包含了所需要的3个值：cnt_n_1 cnt_uniq

In [73]:
# 2.2.3 计算P=cnt_n_1 / cnt_uniq: data_withn_cnt_n12

tmp = data_withn_cnt_n1['cnt_n_1'] / data_withn_cnt_n1['cnt_uniq']
data_withn_cnt_n1['P'] = tmp

In [74]:
# 2.2.4 逻辑检查
# data_withn_cnt_n1.to_csv("data_check_tmp.csv", header=True, encoding="utf-8",index=False)

## 2.3 保存结果
0. 逻辑检查请在之前进行
1. 这里保存的是没有经过`threshold`筛选的data
2. 去重注意不能按照`unique_setting`去搞，毕竟一个`unique_setting`对应一组`np`值，也就是对应一个概率分布

In [75]:
# 去重，重命名与输出
data_withn_cnt_n1.rename(columns={'n_1': 'N'}, inplace=True)
data_withn_cnt_n1.drop_duplicates(subset=['N','product_id','bidincrement','bidfee','P'],inplace=True)
data_withn_cnt_n1.to_csv(output_path, header=True, encoding="utf-8",index=False)
print("The data is like: ")
print(data_withn_cnt_n1.head())

The data is like: 
     N  product_id  bidincrement  bidfee  retail  cnt_uniq  cnt_n_1         P
0  151    10012342            12      60   64.12        12        1  0.083333
1   79    10012342            12      60   64.12        12        1  0.083333
2   72    10012359            12      60  169.99        14        1  0.071429
3  283    10012359            12      60  169.99        14        1  0.071429
4   51    10011778            12      60   27.99         9        1  0.111111


# 3. 根据threshold筛选data
1. 取样本数在threshold之上的setting作为数据集来使用
    - 根据每个unique setting[product_id,bidincrement,bidfee]下的样本数: data_withn_cnt_n1['cnt_uniq']，用`threshold`筛选
2. GTmodel可以根据这个进行计算

In [76]:
# 筛选unique setting对应的样本数(data_withn_cnt_n1['cnt_uniq'])在threshold之上的部分
data_selected = data_withn_cnt_n1[data_withn_cnt_n1['cnt_uniq'] >= threshold][:]
data_selected.to_csv(data_selected_path,header=True,encoding="utf-8",index=False)

# 4. output
1.输出一些信息

In [77]:
total_amount = data_withn_cnt_n1.shape[0]
data_selected_size = data_selected.shape[0]
print("在当前threshold设置下，dataset包括{}个setting\n".format(data_selected_size))
print("当前threshold为{0}，相当于取了{1}%个unique settings\n".format(threshold, round(data_selected_size/total_amount*100, 3)))

在当前threshold设置下，dataset包括3838个setting

当前threshold为16，相当于取了60.412%个unique settings

